In [1]:
import itertools
import pandas as pd
import numpy as np
import sys

In [2]:
class Solver:
    def __init__(self, constraints : list, sign : list, constraint_values : list, df : pd.DataFrame):
        self.constraints = constraints
        self.sign = sign
        self.constraint_values = constraint_values
        self.df = df

    def checkAllConstraints(self, values : pd.Series):
    
        for i in range(len(self.constraints)):
            cur_constraint = (self.constraints[i]).split(" + ")
            # print(cur_constraint)

            LHS = 0
            for j in range(len(cur_constraint)):
                LHS += values[cur_constraint[j]]

            if self.sign[i] == "eq":
                if LHS != self.constraint_values[i]:
                    return False

            elif self.sign[i] == "leq":
                if LHS > self.constraint_values[i]:
                    return False

        return True

    def findValidSolutions(self):
        self.valid_sols = []
        for i in range(self.df.shape[0]):
            valid = self.checkAllConstraints(self.df.loc[i])

            if valid == True:
                self.valid_sols.append(i)
        return self.valid_sols

    def Z(self, row, coeficients):
        values = self.df.loc[row]
        return np.dot(values,coeficients)

    def findOptimumSolution(self, coeficients : list):
        self.findValidSolutions()
        min_ans = sys.maxsize
        for i in range(len(self.valid_sols)):
            cur_ans = self.Z(self.valid_sols[i], coeficients)
            if cur_ans < min_ans:
                min_ans = cur_ans
                row = self.valid_sols[i]
        return self.df.loc[row]

    def printOptimumPath(self, coeficients : list):
        series = self.findOptimumSolution(coeficients)
        path = []
        for i in range(series.shape[0]):
            if series[i] == 1:
                path.append(series.index[i])
        return path

In [3]:
all_pos_sols = pd.DataFrame(list(itertools.product([0, 1], repeat=12)), columns = ["X12", "X13", "X14", "X21", "X23", "X32", "X24", "X42", "X31", "X34", "X43", "X41"])
all_pos_sols.head()

,X12,X13,X14,X21,X23,X32,X24,X42,X31,X34,X43,X41
0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,1,0,0


In [4]:
const = ["X12 + X13 + X14", "X21 + X31 + X41", "X21 + X24 + X23", "X12 + X42 + X32", "X34 + X31 + X32", "X23 + X13 + X43", "X14 + X24 + X34", "X41 + X42 + X43",
         "X12 + X21","X23 + X32","X34 + X43","X14 + X41","X13 + X31","X24 + X42",
         "X12 + X21 + X23 + X32 + X13 + X31","X12 + X21 + X14 + X41 + X24 + X42","X23 + X32 + X34 + X43 + X24 + X42","X13 + X31 + X34 + X43 + X14 + X41"]
sign = ["eq","eq","eq","eq","eq","eq","eq","eq",
         "leq", "leq", "leq", "leq","leq", "leq",
         "leq","leq","leq","leq"]
const_vals = [1,1,1,1,1,1,1,1,
          1,1,1,1,1,1,
          2,2,2,2]

In [5]:
model = Solver(const, sign, const_vals, all_pos_sols)

In [6]:
coef = [15,35,30,15,10,10,15,15,35,20,20,30]

In [7]:
model.printOptimumPath(coef)

['X14', 'X21', 'X32', 'X43']